In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#Importing the basic librarires

import os
import math
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display

#from brokenaxes import brokenaxes
from statsmodels.formula import api
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
import lightgbm as lgb
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10,6]

import warnings 
warnings.filterwarnings('ignore')

In [ ]:
df_train=pd.read_csv("../input/store-sales-time-series-forecasting/train.csv",index_col=0)
df_test =pd.read_csv("../input/store-sales-time-series-forecasting/test.csv",index_col=0)
df_transactions = pd.read_csv("../input/store-sales-time-series-forecasting/transactions.csv")
df_stores = pd.read_csv("../input/store-sales-time-series-forecasting/stores.csv")
df_holidays = pd.read_csv("../input/store-sales-time-series-forecasting/holidays_events.csv")

In [ ]:
df_oil = pd.read_csv("../input/store-sales-time-series-forecasting/oil.csv")
df_oil['date'] = df_oil['date'].astype("datetime64")
df_date = pd.DataFrame(pd.date_range("2013-01-01","2017-08-31"),columns=["date"])
df_oil = df_date.merge(df_oil,left_on='date',right_on='date',how='left').fillna(method='ffill').fillna(method='bfill')

In [ ]:
df_oil = pd.read_csv("../input/store-sales-time-series-forecasting/oil.csv")
df_oil['date'] = df_oil['date'].astype("datetime64")
df_date = pd.DataFrame(pd.date_range("2013-01-01","2017-08-31"),columns=["date"])
df_oil = df_date.merge(df_oil,left_on='date',right_on='date',how='left').fillna(method='ffill').fillna(method='bfill')
df_all = df_train.append(df_test)
df_all = df_all.merge(df_stores,left_on="store_nbr",right_on="store_nbr",how="left")
df_holidays = df_holidays.rename(columns = {"type":"Holiday_type"})
df_all = df_all.merge(df_holidays,left_on="date",right_on="date",how="left")
df_all = df_all.merge(df_transactions,left_on=["date","store_nbr"],right_on=["date","store_nbr"],how="left")
df_all["date"] = df_all["date"].astype("datetime64")
df_all = df_all.merge(df_oil,left_on="date",right_on="date",how="left")
df_all = df_all.replace(",","_",regex=True)
df_all = df_all.rename(columns = {"date":"Date"})

In [ ]:
#Importing the dataset

display(df_all.head())

original_df = df_all.copy(deep=True)

print('\n\033[1mInference:\033[0m The Datset consists of {} features & {} samples.'.format(df_all.shape[1], df_all.shape[0]))

In [ ]:
df_all

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.time_series.ex4 import *

# Setup notebook
from pathlib import Path
from learntools.time_series.style import *  # plot style settings
from learntools.time_series.utils import plot_lags, make_lags, make_leads

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess


comp_dir = Path('../input/store-sales-time-series-forecasting')

store_sales = pd.read_csv(
    comp_dir / 'train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()

family_sales = (
    store_sales
    .groupby(['family', 'date'])
    .mean() 
    .unstack('family')
    .loc['2017', ['sales', 'onpromotion']]
)

In [ ]:
supply_sales = family_sales.loc(axis=1)[:, 'SCHOOL AND OFFICE SUPPLIES']
y = supply_sales.loc[:, 'sales'].squeeze()

fourier = CalendarFourier(freq='M', order=4)
dp = DeterministicProcess(
    constant=True,
    index=y.index,
    order=1,
    seasonal=True,
    drop=True,
    additional_terms=[fourier],
)
X_time = dp.in_sample()
X_time['NewYearsDay'] = (X_time.index.dayofyear == 1)

model = LinearRegression(fit_intercept=False)
model.fit(X_time, y)
y_deseason = y - model.predict(X_time)
y_deseason.name = 'sales_deseasoned'

ax = y_deseason.plot()
ax.set_title("Sales of School and Office Supplies (deseasonalized)");

In [ ]:
plot_pacf(y_deseason, lags=8);
plot_lags(y_deseason, lags=8, nrows=2);

In [ ]:
df=df_all.copy()

In [ ]:
# Reframing the columns

df_all.Date=pd.to_datetime(df_all.Date)

df_all['weekday'] = df_all.Date.dt.weekday
df_all['month'] = df_all.Date.dt.month
df_all['year'] = df_all.Date.dt.year

df_all['Monthly_Quarter'] = df_all.month.map({1:'Q1',2:'Q1',3:'Q1',4:'Q2',5:'Q2',6:'Q2',7:'Q3',
                                       8:'Q3',9:'Q3',10:'Q4',11:'Q4',12:'Q4'})

#df_all.drop(['Date'], axis=1, inplace=True)#,'month'

target = 'sales'
features = [i for i in df.columns if i not in [target]]
#original_df = df_all.copy(deep=True)

df_all.head()

In [ ]:
D=df_all.copy()

In [ ]:

D['family'].unique()

In [ ]:
D.columns

In [ ]:
D[(D["family"]=="BEAUTY") & (D["store_nbr"]==1)].head(5)

In [ ]:
d_beauty=D[(D["family"]=="BEAUTY") ]

In [ ]:
d=D[(D["store_nbr"]==1) & (D['Holiday_type']=='Holiday') & (D["family"]=='BEAUTY') & (D["month"]==1)]

In [ ]:
d

In [ ]:
d_baby=D[(D["family"]=="BABY CARE") &  (D["city"]=='Machala') & (D['Holiday_type']=='Holiday') & (D['sales']>=1) &(D['store_nbr']==41)]

In [ ]:
d_baby

In [ ]:
country_data = d_baby["Monthly_Quarter"]
medal_data = d_baby["sales"]
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#8c564b"]
explode = (0.1, 0, 0, 0, 0)  
plt.pie(medal_data, labels=country_data, explode=explode, colors=colors,
autopct='%1.1f%%', shadow=True, startangle=140)
plt.title(" In BABY CARE family store_nbr : 41 sales report on the basis of month ")
plt.show()

In [ ]:
columns = ['Date','store_nbr', 'family', 'sales', 'onpromotion','cluster', 'Holiday_type', 'locale', 'transferred','description','dcoilwtico',
      ]
df = df[columns]

In [ ]:
df.tail()

In [ ]:
df['family'].unique()

***Process data for building Model***

In [ ]:
data=df.copy()

In [ ]:
data.loc[:,"store_nbr"] = data["store_nbr"].astype("category")
data.loc[:,"family"] = data["family"].astype("category")
data.loc[:,"onpromotion"] = data["onpromotion"].astype("int64")
#data.loc[:,"city"] = data["city"].astype("category")
data.loc[:,"Holiday_type"] = data["Holiday_type"].astype("category")
data.loc[:,"locale"] = data["locale"].astype("category")
data.loc[:,"transferred"] = data["transferred"].astype("category")
zero_index = data.groupby(["store_nbr","family"]).sum()[data.groupby(["store_nbr","family"]).
                                                      sum()["sales"]==0].index

In [ ]:
df=original_df.copy()


In [ ]:
df

In [ ]:
df.head()

In [ ]:
df_all["family"]=df_all["family"].astype("category")
df_all["store_nbr"]=df_all["store_nbr"].astype("category")
df_all["city"]=df_all["city"].astype("category")
df_all["state"]=df_all["state"].astype("category")
df_all["type"]=df_all["type"].astype("category")

In [ ]:
df_all


In [ ]:
df.drop(['transferred'], axis=1, inplace=True)
df.drop(['Holiday_type'], axis=1, inplace=True)
df.drop(['city'], axis=1, inplace=True)
df.drop(['state'], axis=1, inplace=True)
df.drop(['locale'], axis=1, inplace=True)
df.drop(['locale_name'], axis=1, inplace=True)
df.drop(['description'], axis=1, inplace=True)
df.drop(['transactions'],axis=1,inplace=True)
df.drop(['family'],axis=1,inplace=True)
df.drop(['Monthly_Quarter'],axis=1,inplace=True)
df.drop(['type'],axis=1,inplace=True)
df.drop(['sales'],axis=1,inplace=True)

In [ ]:
df

In [ ]:
# Import necessary libraries and make necessary arrangements
import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import xgboost 
import lightgbm as lgb
import warnings
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
warnings.filterwarnings('ignore')

In [ ]:
  df ['sales'] = df ['sales'].fillna (0)

In [ ]:
df= df[df["Date"]>"2013-12-31"]

val = df[(df["Date"]>"2017-08-01")&(df["Date"]<="2017-08-15")]
train  = df[df["Date"]<="2017-08-01"]
test = df[df["Date"]>"2017-08-15"]

col_X = [col for col in train.columns if col not in ['Date', 'sales','year']]

y_train = train["store_nbr"]
y_val = val["store_nbr"]
X_train = train[col_X]
X_val = val[col_X]
X_test = test[col_X]
y_test = test["store_nbr"]

In [ ]:
X_train

In [ ]:
# df=original_df.copy()


In [ ]:
lgb_params = {'metric': {'mse'},
              'boosting_type' : 'gbdt',
              'num_leaves': 8,
              'learning_rate': 0.2,
              #'feature_fraction': 0.8,
              'max_depth': 7,
              'verbose': 0,
              'num_boost_round': 5000,
              'early_stopping_rounds': 200,
              'nthread': -1,
             'force_col_wise':True}

lgbtrain = lgb.Dataset(data=X_train, label=y_train, feature_name=col_X)
lgbtest = lgb.Dataset(data=X_val, label=y_val, reference=lgbtrain, feature_name=col_X)

#del Y_train, X_train, Y_test,X_test

In [ ]:
# df.drop(['family'],axis=1,inplace=True)

In [ ]:
from sklearn.metrics import mean_absolute_error
model = lgb.train(lgb_params, lgbtrain,
                  valid_sets=[lgbtrain, lgbtest],
                  num_boost_round=lgb_params['num_boost_round'],
                  early_stopping_rounds=lgb_params['early_stopping_rounds'],
                  #feval=mean_absolute_error,
                  verbose_eval=100,
                  )

In [ ]:
y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)

In [ ]:
from sklearn.metrics import mean_squared_log_error

In [ ]:
y_pred_val[y_pred_val<0]=0

In [ ]:
mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))

In [ ]:
plt.scatter(np.expm1(y_val),np.expm1(y_pred_val))

In [ ]:
model.feature_importance()

In [ ]:
model.feature_name()

***Submission***

In [ ]:
submission = pd.read_csv("../input/store-sales-time-series-forecasting/sample_submission.csv")
y_test = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
y_test[y_test<0]=0

In [ ]:
submission["sales"] = np.expm1(y_test)

In [ ]:
submission.to_csv("submission.csv",index=False)

In [ ]:
submission